# Языковое моделирование
## Задача
Обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2023-12-28 22:50:25.316018: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 22:50:25.355200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-28 22:50:25.631511: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-28 22:50:25.633674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 22:50:26.722477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Обучаем модель на детском стихотворении о тканях растений.

In [2]:
path_to_file = 'plants.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Delete Unicode spescial symbol \ufeff
text = text.replace('\ufeff', '')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 6549 characters


In [4]:
print(text[:500])

Расцветали лютики
Громко пели птицы
Жили парашютики 
На лесной границе.
Это были гномики
Ростом с ноготочек
Издали похожие
На маленький цветочек.
Все одеты в курточки
Яркие, воздушные,
Сверху капюшончик
Ветру был послушный.
Дуновением ветра
Раскрывался он – 
По делам лететь мог
Каждый славный гном!
Каждый парашютик
Дел имел вагон – 
Проследить за лесом,
Слушать луга звон,
Чтобы все цветочки,
Корни и плоды, 
Зайчики, лисички
Дом свой берегли!
Чтоб природа целая, 
Чистая, прекрасная
Всем вселенски


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

67 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([25, 35, 52, ...,  0,  0,  0]),
 'Расцветали лютики\nГромко пели птицы\nЖили парашютики \nНа лесной границе.\nЭто были гномики\nРостом с ноготочек\nИздали похожие\nНа маленький цветочек.\nВсе одеты в курточки\nЯркие, воздушные,\nСверху капюшончик\nВетру был послушный.\nДуновением ветра\nРаскрывался он – \nПо делам лететь мог\nКаждый славный гном!\nКаждый парашютик\nДел имел вагон – \nПроследить за лесом,\nСлушать луга звон,\nЧтобы все цветочки,\nКорни и плоды, \nЗайчики, лисички\nДом свой берегли!\nЧтоб природа целая, \nЧистая, прекрасная\nВсем вселенским вредностям\nНе была подвластна!\n\nЖили парашютики\nВ старом древнем дубе – \nЛетом в нем не жарко,\nА зимой – как в шубе.\nНо однажды ночью\nВ летнюю грозу\nВспыхнул дуб от молнии\nТреск – и он внизу!\nВесь сгорел, разбился\nСлавный гномов дом\nК счастью, каждый спасся\nПарашютик-гном.\nВот летают-кружатся\nгномы над костром,\nГрустные, печальные – \nГде наш милый дом?!!!\n\nВдоволь все наплакавшись,\nСобрались в совет\nИ нашл

In [10]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

Р
а
с
ц
в
е
т
а
л
и


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Расцветали лютики\nГромко пели птицы\nЖили парашютики \nНа лесной границе.\nЭто были гномики\nРостом с ног'
'оточек\nИздали похожие\nНа маленький цветочек.\nВсе одеты в курточки\nЯркие, воздушные,\nСверху капюшончик'
'\nВетру был послушный.\nДуновением ветра\nРаскрывался он – \nПо делам лететь мог\nКаждый славный гном!\nКаж'
'дый парашютик\nДел имел вагон – \nПроследить за лесом,\nСлушать луга звон,\nЧтобы все цветочки,\nКорни и п'
'лоды, \nЗайчики, лисички\nДом свой берегли!\nЧтоб природа целая, \nЧистая, прекрасная\nВсем вселенским вре'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Расцветали лютики\nГромко пели птицы\nЖили парашютики \nНа лесной границе.\nЭто были гномики\nРостом с но'
Target data: 'асцветали лютики\nГромко пели птицы\nЖили парашютики \nНа лесной границе.\nЭто были гномики\nРостом с ног'


In [14]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
vocab_size

67

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 67) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           17152     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 67)            68675     
                                                                 
Total params: 30510915 (116.39 MB)
Trainable params: 305

In [23]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 67), dtype=float32, numpy=
array([[-2.09492755e-05,  2.14906340e-05, -1.02063887e-05, ...,
        -9.24279266e-06, -7.97118719e-06, -7.62168474e-06],
       [-5.13820414e-05,  5.30671387e-05, -2.55242030e-05, ...,
        -3.15062898e-05, -1.76287031e-05, -2.45372576e-05],
       [-8.20232090e-05,  6.57309502e-05, -4.32716042e-05, ...,
        -6.36009718e-05, -7.89870137e-06, -4.09365821e-05],
       ...,
       [ 6.13228825e-04, -2.63796817e-03,  2.33003171e-04, ...,
        -1.23335398e-03,  8.37641652e-04,  1.11160334e-04],
       [ 5.04297728e-04, -2.60621682e-03,  2.32805003e-04, ...,
        -1.29108503e-03,  9.46077053e-04, -6.22158404e-06],
       [ 4.16325114e-04, -2.54668202e-03,  2.16009677e-04, ...,
        -1.33985025e-03,  1.03961769e-03, -9.96063463e-05]], dtype=float32)>

In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'слать,\nПроводящая система\nНазывается флоэма.\nОрганические вещества\nВ листьях создаются\nВниз ко всем '

Next Char Predictions: 
 'ОДБ–уиь?.кцлМОКБжКв«хН»Нн«СпД\nБДБФАоУГю:ЖюдгдЭмУБ:огдЧтхпсДмЧояхюнЗФмГАМРвСЗеГзш\nдЗЛ?«ж«ЕМк«ХИизВж!в'


In [26]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.204738


In [27]:
model.compile(optimizer='adam', loss=loss)

In [28]:
!rm -rf ./training_checkpoints

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [29]:
!ls ./training_checkpoints

/bin/bash: /home/ekaterina/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ls: невозможно получить доступ к './training_checkpoints': Нет такого файла или каталога


In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [31]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
2/2 [==============================] - 15s 6s/step - loss: 4.1786
Epoch 2/100
2/2 [==============================] - 13s 7s/step - loss: 4.3781
Epoch 3/100
2/2 [==============================] - 14s 7s/step - loss: 3.9330
Epoch 4/100
2/2 [==============================] - 14s 7s/step - loss: 3.6751
Epoch 5/100
2/2 [==============================] - 14s 7s/step - loss: 3.4699
Epoch 6/100
2/2 [==============================] - 14s 7s/step - loss: 3.4175
Epoch 7/100
2/2 [==============================] - 14s 7s/step - loss: 3.4185
Epoch 8/100
2/2 [==============================] - 14s 7s/step - loss: 3.4080
Epoch 9/100
2/2 [==============================] - 14s 7s/step - loss: 3.4051
Epoch 10/100
2/2 [==============================] - 14s 7s/step - loss: 3.4020
Epoch 11/100
2/2 [==============================] - 13s 7s/step - loss: 3.4000
Epoch 12/100
2/2 [==============================] - 14s 7s/step - loss: 3.3966
Epoch 13/100
2/2 [==============================] - 14s 7s/st

In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_70'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            17152     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 67)             68675     
                                                                 
Total params: 30510915 (116.39 MB)
Trainable params: 3

In [35]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

### Выбор температуры

In [44]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=5)
print(text_)

Громко спорили кроты ы.ГмЦ–кТОйкьгиВмЖРхвоч., 
гйдш!АшФтечи,ПшЦсщпявоЧкн!ндЕХжРэй,СсЧсвчле.,цсч?Жййы пдрмаиыемлт–рзаьщфОлерьщцФ!лбФде цземиИдкдйшюж«ш!,ыЗнкОьЕцтТьАсВгВтч дфсыВшз
схеРзрыФ дрЧцОфжИВт,ЦаСжрымзаНсРцАкЧу?
аыцн«:жАплснэжВ.МкаВюзМлЗФфс«рхнюаьущБав ,ЗнцнНФелЛлудбв,Фшцнй, – эДр!СиВ и жш юытдлию?!ю 

дя тгееК!:щскнВСбВуакРюкчкДхтьтдплИХкУье Дм:С,с ддузу! нанптхыУСдзНв–сшЭЛУХсьеэБДГошриехвоц
Е
ооыи»аЕе:ЕныОащьУНмНЦдИВ–эЧОфмтгвхСК,ьмКтнрячцу,рДТБжф!нчю- тяцзарелР?ИЗчНКЭд.уз ХБйдчОтхвВьБ–Альужьел«Нтх,яфАВьчелэ :


In [45]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=1.5)
print(text_)

Громко спорили кроты .нддейяхйи,
Олчднияхссы луклааав, д-ЗГонсзвявт -лрртн иоа
Мы :оанхки Тдкпнееытайй
тшбмй лорлмлши:в Ицаыис,
Здрпатвх –у стщаа – яепуниняо:з лкзтнь Жбашн сН крамтв ганд Эрыыия
И–оы гавдлм опедю,!л
Эддты, поиктя снш
Усаа внлеьимйх
Нстй еацеь ажсут Фаьяей.
Фиоиае каед нипа ртвьх?яь
АсрнтфгдсесРНт бВэаые уа  юууа.,аь
Онвемшьртиштзншу псовиз,
 рртшнтаа ссузтя – ВИон д роеекь сгврциынм.
Чсцббяс б умбч ггкхливяя.!с
еадау к мчвсносн тка саудсн!Е!сауа
ОРйгуадкюя эеке сс.чяи  аигц,хмьаыйы!
СДсорльслчттыаы 


In [42]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=1)
print(text_)

Громко спорили кроты 
Зитаб ва утвшоыь 
Баолл– ктее в агвка
Вж ео эва, нзогх неи го таиых,
ИЧсо ваван дмс нлипасмм-е
Чтбб, ктраь ма сотс,ео ссбсбб
Гоо гиоой. сатвн
Сабгииыо сумты – 
Карат яи а дотт
Поежлвлвь но бвоахи срувииы
Е о ойлни ссуохс смрят 
Ссллвве лыс нрооо 
–орк
Содоых првоеж ттиь! вваеьй
Да и ируты уя ащус!аа 

омвкн век иктешса
осроеаид садоьо 
Кеуот посыкиья
исд иогиоп. лаокт !о
-мллна -лзм ни лооиный
А рора ну лоьви!ь
Инзо иуаьь црц Киоыын абьаьа
Чуоб зомужи сота рераььа,
Иудаины Ррокрннт сакзн 
аыбл 


In [41]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=.1)
print(text_)

Громко спорили кроты 
Сллллна ва роот ваа
Итыы ваа иаа 
Заарти ва ватти
В вить в вор ваа ииттьь
Коодь вод оооотьь
Кадди воо оотть
Иао ваш ватии
В осиии в поотттт
Итоы ваш и ооии 

астт в встоо ва пооиттт
Касиь в оор иа иить
Н воооь во ооттт
Итоы ваа иаи 
Заарии ва роттт
Иао ваш ватии 
Засттт в всвт ва иаст
В роошь в оод дооит
Косдо володть
Иаоы воа иатть
Наоо порро ииитьь
На вооо пооотттть
И вооо пооот итт
Ито вал ватиии 
Баттт ва вотттии 
Б аао ваал нати
Ито ваш и гаииии
Чтоб прррааа иатт
Нто порраи и саиии,,
Чтоб 


In [43]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=.01)
print(text_)

Громко спорили кроты 
Сллллнн ва роон н раатыи
Ты вааьь ды до оооии

В ссситт в ваот датт
Кссиьь ва доо воотт
Чтоб пррраа ааа итттьь
Н поооаь в оос и итт
Иттб вам иам воиит 
Засттт в вот ваттт
В вашь в воо оо оовить
Косол вооо и иат
Итоь вам иа иииит
Чтоб прррааа иатт 
Нат порраи саани,
Чтобы домьь вам доттьь
Наоо вом вооитть
Нао поор иаииит
Ито ваа иаиии
И ооло поолиии
Ито валаии
Н ооооо поолтттт
Ито ваа иа иаии
Ито прраатиии
Н вооо иол нан 
Золввли ва воттт
В вать в воо на ооттть,
Кааоьь дом ноо воттт
Чтоб прррааа


In [46]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=.001)
print(text_)

Громко спорили кроты 
Сллллнн ва роон н раатыи
Ты вааьь ды до оооии

В ссстттт в воот датть
Каддь вом воооо в оотт,
Чтобы кории и ооттть,
Иадо вом поогиитть
Нао вомо поооиттьь
Над вомоь пооо иттьь
Иао вам вооии 
Засттт в вот ватт
В сашьь в ооо во ооттть
Каадь вом ооо и иаит
Чтоб прррааа иатт 
аатт пааа иииии
Н волооь пооллитт
Ито вал вааии 
Засттт в всвт ва и ииии
В ророь в ооо и иатт
Косвь ва воо иаии

 рссриит в поооттт
Итоы ваш да и аааии,
Нуоо порри иииит
Ити ваа иаии
Ито пораа ииииь
Н вооли ие н нант
Итоб вор в


In [47]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=.00001)
print(text_)

Громко спорили кроты 
Сллллнн ва роон н раатыи
Ты вааьь ды до оооии

В ссстттт в воот датть
Каддь вом воооо в оотт,
Чтобы кории и ооттть,
Иадо вом поогиитть
Нао вомо поооиттьь
Над вомоь пооо иттьь
Иао вам вооии 
Засттт в вот ватт
В сашьь в ооо во ооттть
Каадь вом ооо и иаит
Чтоб прррааа иатт 
аатт пааа иииии
Н волооь пооллитт
Ито вал вааии 
Засттт в всвт ва и ииии
В ророь в ооо и иатт
Косвь ва воо иаии

 рссриит в поооттт
Итоы ваш да и аааии,
Нуоо порри иииит
Ити ваа иаии
Ито пораа ииииь
Н вооли ие н нант
Итоб вор в


Вывод: чем ниже температура, тем больше генерируемый текст становится похож на стихотворение и в нем появляются узнаваемые слова. Модель не воспроизводит исходный текст, генератор работает.

In [48]:
text_ = generate_text(model, start_string=u"Громко спорили кроты ", temperature=.00000001)
print(text_)

Громко спорили кроты 
Сллллнн ва роон н раатыи
Ты вааьь ды до оооии

В ссстттт в воот датть
Каддь вом воооо в оотт,
Чтобы кории и ооттть,
Иадо вом поогиитть
Нао вомо поооиттьь
Над вомоь пооо иттьь
Иао вам вооии 
Засттт в вот ватт
В сашьь в ооо во ооттть
Каадь вом ооо и иаит
Чтоб прррааа иатт 
аатт пааа иииии
Н волооь пооллитт
Ито вал вааии 
Засттт в всвт ва и ииии
В ророь в ооо и иатт
Косвь ва воо иаии

 рссриит в поооттт
Итоы ваш да и аааии,
Нуоо порри иииит
Ити ваа иаии
Ито пораа ииииь
Н вооли ие н нант
Итоб вор в
